<a href="https://colab.research.google.com/github/Ani-Das/MachineLearning_MajorProject_Messageclassifier/blob/main/SpamDetectionMLmajorproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np

#Source
dtfrm = pd.read_table('https://raw.githubusercontent.com/Ani-Das/Machine-Learning-using-sklearn/master/Datasets/spam.tsv')
dtfrm.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [2]:
dtfrm['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [26]:
#Data cleaning and preprocessing

dtfrm['message'] = dtfrm['message'].str.lower()
dtfrm.head()

,label,message,length,punct
0,ham,"go until jurong point, crazy.. available only ...",111,9
1,ham,ok lar... joking wif u oni...,29,6
2,spam,free entry in 2 a wkly comp to win fa cup fina...,155,6
3,ham,u dun say so early hor... u c already then say...,49,6
4,ham,"nah i don't think he goes to usf, he lives aro...",61,2


In [27]:
#TFIDF Vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
sent = [dtfrm['message'][0],dtfrm['message'][1],dtfrm['message'][2],dtfrm['message'][3],dtfrm['message'][4]]
a = vect.fit_transform(sent)

z = a.toarray()
df = pd.DataFrame(z,columns = vect.get_feature_names())
df.head()

,08452810075over18,2005,21st,87121,already,amore,apply,around,available,buffet,bugis,cine,comp,crazy,cup,don,dun,early,entry,fa,final,free,go,goes,got,great,he,here,hor,in,joking,jurong,la,lar,lives,may,nah,ok,oni,only,point,question,rate,receive,say,so,std,text,then,there,think,though,tkts,to,txt,until,usf,wat,wif,win,wkly,world
0,0.00000,0.00000,0.00000,0.00000,0.000000,0.238022,0.00000,0.000000,0.238022,0.238022,0.238022,0.238022,0.00000,0.238022,0.00000,0.000000,0.000000,0.000000,0.0000,0.0000,0.00000,0.00000,0.238022,0.000000,0.238022,0.238022,0.000000,0.000000,0.000000,0.192034,0.000000,0.238022,0.238022,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.238022,0.238022,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.238022,0.000000,0.000000,0.00000,0.000000,0.00000,0.238022,0.000000,0.238022,0.000000,0.00000,0.00000,0.238022
1,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.0000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.447214,0.000000,0.00000,0.000000,0.447214,0.447214,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.447214,0.00000,0.00000,0.000000
2,0.17275,0.17275,0.17275,0.17275,0.000000,0.000000,0.17275,0.000000,0.000000,0.000000,0.000000,0.000000,0.17275,0.000000,0.17275,0.000000,0.000000,0.000000,0.3455,0.3455,0.17275,0.17275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.139374,0.000000,0.000000,0.000000,0.000000,0.000000,0.17275,0.000000,0.000000,0.000000,0.000000,0.000000,0.17275,0.17275,0.17275,0.000000,0.000000,0.17275,0.17275,0.000000,0.000000,0.000000,0.000000,0.17275,0.418121,0.17275,0.000000,0.000000,0.000000,0.000000,0.17275,0.17275,0.000000
3,0.00000,0.00000,0.00000,0.00000,0.316228,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.316228,0.316228,0.0000,0.0000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.316228,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.632456,0.316228,0.00000,0.00000,0.316228,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
4,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.270657,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.270657,0.000000,0.000000,0.0000,0.0000,0.00000,0.00000,0.000000,0.270657,0.000000,0.000000,0.541314,0.270657,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.270657,0.00000,0.270657,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.270657,0.270657,0.00000,0.218364,0.00000,0.000000,0.270657,0.000000,0.000000,0.00000,0.00000,0.000000


In [31]:
x = dtfrm.iloc[:,1]
y = dtfrm.iloc[:,0]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0,stratify=y)
print(x_train.shape)
print(x_test.shape)
np.unique(y_train,return_counts=True)

(3900,)
(1672,)


(array(['ham', 'spam'], dtype=object), array([1448,  224]))

In [34]:
#Pipeline

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
train_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())])

train_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', d

In [36]:
y_pred = train_model.predict(x_test)
y_pred

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [37]:
y_test

313     ham
2241    ham
3024    ham
3724    ham
2644    ham
       ... 
5279    ham
3439    ham
1185    ham
5539    ham
4847    ham
Name: label, Length: 1672, dtype: object

In [38]:
# Prediction model evaluation (Accuracy score, confusion matrix and classification report)

from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
accuracy_score(y_pred,y_test)

0.9778708133971292

In [39]:
confusion_matrix(y_pred,y_test)

array([[1446,   35],
       [   2,  189]])

In [41]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         ham       1.00      0.98      0.99      1481
        spam       0.84      0.99      0.91       191

    accuracy                           0.98      1672
   macro avg       0.92      0.98      0.95      1672
weighted avg       0.98      0.98      0.98      1672



In [42]:
# Testing model

train_model.predict(["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply "])

array(['spam'], dtype=object)

In [48]:
# Dump and load
import joblib
joblib.dump(train_model,'Message_classifier')

['Message_classifier']

In [61]:
# Web app 

!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [66]:
%%writefile clm_app.py
import streamlit as st
import joblib
model = joblib.load('Message_classifier')
st.title('Spam Ham Message Classifier')
ip=st.text_input('Enter your message')
op = model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Writing clm_app.py


In [70]:
!nohup streamlit run clm_app.py &
#link=ngrok.connect(port='8501')
#link

nohup: appending output to 'nohup.out'
